<a href="https://colab.research.google.com/github/siddhantjain/disentangled_rnns/blob/main/disentangled_rnns/notebooks/train_single_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/google-deepmind/disentangled_rnns/blob/main/disentangled_rnns/notebooks/train_single_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install disentangled_rnns repo from github
!git clone https://github.com/google-deepmind/disentangled_rnns
%cd disentangled_rnns
!pip install .
%cd ..


import optax
import matplotlib.pyplot as plt
import matplotlib as mpl
import haiku as hk

from disentangled_rnns.library import rnn_utils
from disentangled_rnns.library import get_datasets

# Define a dataset

In [ ]:
dataset = get_datasets.get_q_learning_dataset(n_sessions=500, n_trials=200)
dataset_train, dataset_eval = rnn_utils.split_dataset(dataset, eval_every_n=2)

# Define and train RNN

In [ ]:
# Define the architecture of the network we'd like to train
n_hidden = 16
output_size = 2

def make_network():
  model = hk.DeepRNN(
      [hk.GRU(n_hidden), hk.Linear(output_size=output_size)]
  )
  return model

In [ ]:

# INITIALIZE THE NETWORK
# Running rnn_utils.train_network with n_steps=0 does no training but sets up the
# parameters and optimizer state.
optimizer = optax.adam(learning_rate=1e-3)

params, opt_state, losses = rnn_utils.train_network(
    make_network = make_network,
    training_dataset=dataset_train,
    validation_dataset=dataset_eval,
    opt = optimizer,
    loss="categorical",
    n_steps=0)


In [ ]:
# Count Params:
import jax
actual_param_count = sum(x.size for x in jax.tree_util.tree_leaves(params))


def calculate_gru_params(n_h, n_in):
    # Standard formula: 3 * Nh * (Nh + Nin + 1)
    return 3 * n_h * (n_h + n_in + 1)

def calculate_linear_layer_params(n_in, n_out):
    return n_in*n_out + n_out

expected_first_layer_count = calculate_gru_params(n_h=n_hidden, n_in=2)

# Subsequent layers use n_hidden as their input
expected_linear_layer_count = calculate_linear_layer_params(n_in=n_hidden, n_out=2)

print(f'Actual param count: {actual_param_count}')
print(f'Expected param count: {expected_first_layer + expected_linear_layer_count}')


In [ ]:
# TRAIN THE NETWORK
# Running this cell repeatedly continues to train the same network.
# The cell below gives insight into what's going on in your network.
# If you'd like to reinitialize the network and start over, re-run the above cell

n_steps = 1000
optimizer = optax.adam(learning_rate=1e-3)

params, opt_state, losses = rnn_utils.train_network(
    make_network = make_network,
    training_dataset=dataset_train,
    validation_dataset=dataset_eval,
    loss="categorical",
    params=params,
    opt_state=opt_state,
    opt = optimizer,
    loss_param = 1,
    n_steps=n_steps,
    do_plot = True)


In [ ]:
# Run forward pass on the unseen data
xs_eval, ys_eval = dataset_eval.get_all()
network_output, network_states = rnn_utils.eval_network(make_network, params, xs_eval)

# Compute normalized likelihood
score = rnn_utils.normalized_likelihood(ys_eval, network_output)
print(f'Normalized Likelihood: {100*score:.1f}%')

# Plot network activations on an example session
example_session = 0
plt.plot(network_states[:,example_session,:])
plt.xlabel('Trial Number')
plt.ylabel('Network Activations')